In [3]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
import requests

In [55]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#options.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
#     driver = webdriver.Firefox()


Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


<ipython-input-55-5315509cced7>:7: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=options)


In [56]:
driver.get('https://hometimeksa.com/products/%D9%83%D9%88%D8%A8-%D8%B2%D8%AC%D8%A7%D8%AC-%D8%A8%D8%B4%D9%83%D9%84-%D8%B9%D8%B5%D8%B1%D9%8A')
list_colors = driver.find_elements_by_xpath('//ul[@name="الحجم "]//li[@onclick="productOptionListItemClicked(event)"]')

for color in list_colors:
    color.click()
    print(color.text.strip())
    time.sleep(3)
 


كبير
صغير


In [31]:
text = driver.find_element_by_xpath('//div[@class="app d-flex flex-column visible"]').get_attribute('innerHTML')
soup = BeautifulSoup(text, 'html.parser')


In [29]:
def return_element(name, soup):
    try:
        ele = soup.find('p', text=re.compile(f'{name}')).text.replace(name, '').strip()
    except:
        ele = None
    return ele

In [37]:
ts_dimensions_height = return_element('الطول:', soup)
diameter = return_element('القطر:', soup)
base_size = return_element('قطر القاعدة:', soup)
amper = return_element('حجم البطارية :', soup)
rechargeable = return_element('مدة شحن المصباح :', soup)
product_accessories = return_element('الملحقات :', soup)
usage_time = return_element('وقت الاستخدام:', soup)
amper, rechargeable, product_accessories, usage_time

('1800 مل امبير', '3-4 ساعات', 'سلك USB', '12-15ساعه تقريبا')

In [53]:
def scrap_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    products = soup.find_all('div', {'class': 'product-item'})
    link_product = ['https://hometimeksa.com' + p.find('a')['href'] for p in products]
    cats = soup.find('ol', {'class': 'breadcrumb'}).find_all('li')
    len(cats)
    cat1 = cats[1].text.strip()
    data = []
    for p in link_product:
        data.append( {
            'url':p,
            'cat1': cat1,
            })
    return data

In [20]:
list_urls = [
    'https://hometimeksa.com/categories/349798/%D9%81%D8%A7%D8%B2%D8%A7%D8%AA',
    'https://hometimeksa.com/categories/349572/%D9%83%D9%88%D8%B3%D8%AA%D8%B1%D8%A7%D8%AA',
    'https://hometimeksa.com/categories/335852/%D8%A7%D9%84%D9%85%D9%86%D8%AA%D8%AC%D8%A7%D8%AA-%D8%A7%D9%84%D9%85%D8%AE%D9%81%D8%B6%D8%A9',
    'https://hometimeksa.com/categories/323100/%D8%A5%D9%86%D8%A7%D8%B1%D8%A7%D8%AA',

    
]

In [24]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/hometimeksa/Hometimeksa.url_model.xlsx')
for i, url in enumerate(list_urls):
    print('Count: ', i)
    print('URL: ', url)
    data = scrap_url(url)
    df1 = pd.DataFrame(data)
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('Hometime_url_list.xlsx')





Count:  0
URL:  https://hometimeksa.com/categories/349798/%D9%81%D8%A7%D8%B2%D8%A7%D8%AA
Count:  1
URL:  https://hometimeksa.com/categories/349572/%D9%83%D9%88%D8%B3%D8%AA%D8%B1%D8%A7%D8%AA
Count:  2
URL:  https://hometimeksa.com/categories/335852/%D8%A7%D9%84%D9%85%D9%86%D8%AA%D8%AC%D8%A7%D8%AA-%D8%A7%D9%84%D9%85%D8%AE%D9%81%D8%B6%D8%A9
Count:  3
URL:  https://hometimeksa.com/categories/323100/%D8%A5%D9%86%D8%A7%D8%B1%D8%A7%D8%AA


In [ ]:
,  , special_price, , images, , , 

In [54]:
r = requests.get('https://hometimeksa.com/products/%D9%83%D9%88%D8%A8-%D8%B2%D8%AC%D8%A7%D8%AC-%D8%A8%D8%B4%D9%83%D9%84-%D8%B9%D8%B5%D8%B1%D9%8A')
soup = BeautifulSoup(r.text, 'html.parser')


In [51]:
size = driver.find_elements_by_xpath('//ul[@name="الحجم "]//li[@onclick="productOptionListItemClicked(event)"]')
if len(size) > 1:
        for sz in size:
            sz.click()
            time.sleep(0.5)
            print(sz)

coucou


In [39]:
name = soup.find('section').find('h1').text.strip()
price = soup.find('h1', {'class': 'product-formatted-price theme-text-primary'}).text.replace('ر.س', '').strip()
sku =  soup.find('div', {'class': 'product-sku'}).text.strip()
qty = len(soup.find('select', {'id': 'product-quantity'}).find_all('option'))
description = soup.find('div', {'class':'description-paragrah text-justify description-text-clear'}).text.replace('xa0', ' ').strip()
description

'وصف المصباح :الارتفاع : 30 سم\xa0حجم البطارية : 1800 مل امبير\xa0كفاءة الضوء : 80 واط\xa0وقت الاستخدام: 12-15ساعه تقريبا\xa0مدة شحن المصباح : 3-4 ساعاتالملحقات : سلك USB'

In [41]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/promise/Promise_product_test.xlsx')

In [42]:
cat1 = df['categories1'].unique().tolist()
len(cat1)

15

In [44]:
cat2 = df['categories2'].unique().tolist()
len(cat2)

24

In [46]:
len(cat1)

24

In [45]:
for i in range(15, 24):
    cat1.append('')

In [50]:
cats = []
for i in range(24):
    cats.append({'categories1': cat1[i], 'categories2': cat2[i]})

In [51]:
df1 = pd.DataFrame(cats)
df1.to_excel('/home/wafistos/Documents/Projects/scaping_wafi/promise/Promise_cats.xlsx')

In [3]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/cascadesco/Cascadesco.updating.test3.xlsx')
toto = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/promise/categories/Promise_cats.xlsx')

df['categories'] = ''
list_cats = []
for index, row in toto.iterrows():
    list_cats.append({'id': row['id'],  'categories1': row['categories1'], 'categories2': row['categories2'], 'categories3': row['categories3'], })
    
for cat in list_cats:
    df.loc[(df['categories2']== cat['categories2']) & (df['categories3']== cat['categories3']), 'categories'] = cat['id']
    

KeyError: 'categories2'

In [4]:
df.columns

Index(['Unnamed: 0', 'sku', 'name', 'price', 'product_size', 'categories1',
       'categories', 'free_colors', 'raw_materials', 'manufacturer',
       'list_colors', 'link_url', 'description', 'pricipal_img', 'base_image',
       'additionnel_images'],
      dtype='object')